In [ ]:
import csv
import numpy as np
import pandas as pd

## ETF Data Handling

In [2]:
import datetime
import time

In [3]:
"""
# TODO: 
Get those etf name which exists before 12/31/2015
It should be:
    
etf_name_list = ['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH', 'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP', 'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD', 'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI', 'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP', 'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ', 'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE', 'DDG']
len(etf_name_list) == 66
    
try it !!
"""

etf_name_list = []
df = pd.read_csv("Energy Equity ETF List (80).csv")
date = { "name": df["Symbol"], "date": df["Inception"] }
date = pd.DataFrame(date)
#print(date.head())

#轉換時間格式
for a in range(0,len(date)):
    t = time.strptime(date["date"][a], "%d/%m/%Y")
    date["date"][a] = time.strftime("%Y-%m-%d", t)
#print(date.head()) 

#篩選日期
mask = date["date"] <= "2015-12-31"
etf_name_list = list(date[mask]["name"])
    
print(etf_name_list)
len(etf_name_list)

['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH', 'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP', 'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD', 'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI', 'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP', 'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ', 'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE', 'DDG']


66

## Web Crawler

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import requests


start_day = 1451577600       # Dec 31, 2015
#end_day = int(time.time())   # Now()
end_day = 1552665600  #Mar 15, 2015
per_day = 86400
month_trans_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

my_options = Options()
my_options.add_argument("--headless")
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=my_options)

def get_statistics(etf_name):
    statistic = []
    time_list = []
    period2 = end_day
    #period1 = start_day
    period1 = end_day - per_day * 90
    while True:
        break_flag = False
        if period1 <= start_day:
            period1 = start_day
            break_flag = True
        url = 'https://finance.yahoo.com/quote/%s/history?period1=%d&period2=%d&interval=1d&filter=history&frequency=1d'%(etf_name, period1, period2)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features='lxml')
        tr_list = soup.find_all('tr', {"class": "BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)"})
        for tr in tr_list:
            td_list = tr.find_all('td', {'class': 'Py(10px) Pstart(10px)'})
            if len(td_list) < 1:
                continue
            td_time = tr.find('td', {'class': 'Py(10px) Ta(start) Pend(10px)'})
            td_time = re.split(r'[,\s]', td_time.get_text())
            time_format = '%04d-%02d-%02d'%(int(td_time[3]), month_trans_dict[td_time[0]], int(td_time[1]))
            time_list.append(time_format)
            statistic.append(td_list[4].get_text())
        if break_flag:
            break
        period2 = period1 - per_day
        period1 = period2 - per_day * 90
        #print('-----')
    #print(len(statistic), len(time_list))
    statistic.reverse()
    time_list.reverse()
    df = pd.DataFrame({'Date': time_list, etf_name: statistic})
    return df

def get_FENY():
    '''
    #my_options = Options()
    #my_options.add_argument("--headless")
    #driver = webdriver.Chrome('./chromedriver.exe', chrome_options=my_options)
    #driver = webdriver.Chrome('chromedriver.exe')
    driver.get("https://ETF.com/FENY/#overview")
    driver.find_element_by_link_text("Fund Home Page").click()
    driver.find_element_by_id("priceHistory").click()
    driver.find_element_by_link_text("Download to CSV file").click()
    time.sleep(6)
    #driver.quit()
    print("Successfully download FENY \n")
    '''
    #dateframe
    df = pd.read_csv("FENY_Price_History_20190315.csv")
    df = df[550:1356]
    df = df.iloc[:,[0,4]]
    #df = df.iloc[::-1]
    df = df.rename(columns={'Close':'FENY'})
    for a in range(550,1356):
        t = time.strptime(df["Date"][a], "%m/%d/%Y")
        df["Date"][a] = time.strftime("%Y-%m-%d", t)

    FENY = pd.DataFrame(df)
    return FENY

def get_TPYP(): 
    r = requests.get("http://etp.tortoiseadvisors.com/tpyp/premium-discount/")
    if r.status_code == requests.codes.ok:
        soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('table')
    df = pd.read_html(str(tags[0]))
    dff = pd.DataFrame(df[0])
    for a in range(0,904):
        t = time.strptime(dff["Date"][a], "%m/%d/%Y")
        dff["Date"][a] = time.strftime("%Y-%m-%d", t)

    a = dff[0:781] #date filter
    TPYP = a.iloc[:,[0,1]] #keep date & NAV
    TPYP = TPYP.iloc[::-1]
    TPYP = TPYP.rename(columns={'Share Price':'TPYP'})
    return TPYP


In [12]:
def get_all_statistics(etf_name_list):
    statistics = []
    dataFrame_time = []
    dataFrame_column = []
    #etf_name_list = ['XLE', 'TPYP', 'FENY']
    df_all = []
    empty = True
    for etf_name in etf_name_list:
        print('#### start handle %s ####'%(etf_name))
        if etf_name == 'FENY':
            df = get_FENY()
        #elif etf_name == 'TPYP':
        #    df = get_TPYP()
        else:
            if empty:
                df_all = get_statistics(etf_name)
                empty = False
                continue
            else:
                df = get_statistics(etf_name)
        df_all = pd.merge(df_all, df, on='Date', how='outer')
    return df_all

## Main

In [13]:
df_all = get_all_statistics(etf_name_list)
print(df_all)
df_all.to_pickle('df_all.h5')

#### start handle XLE ####
#### start handle AMLP ####
#### start handle VDE ####
#### start handle AMJ ####
#### start handle EMLP ####
#### start handle XOP ####
#### start handle MLPI ####
#### start handle IXC ####
#### start handle OIH ####
#### start handle MLPA ####
#### start handle IYE ####
#### start handle MLPX ####
#### start handle FENY ####


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### start handle AMZA ####
#### start handle ERX ####
#### start handle ATMP ####
#### start handle IEO ####
#### start handle TPYP ####
#### start handle AMU ####
#### start handle TAN ####
#### start handle GUSH ####
#### start handle GREK ####
#### start handle XES ####
#### start handle RYE ####
#### start handle FXN ####
#### start handle ICLN ####
#### start handle IEZ ####
#### start handle PZD ####
#### start handle PBW ####
#### start handle DIG ####
#### start handle FCG ####
#### start handle MLPC ####
#### start handle QCLN ####
#### start handle GEX ####
#### start handle FAN ####
#### start handle KOL ####
#### start handle ENFR ####
#### start handle PXI ####
#### start handle ZMLP ####
#### start handle PBD ####
#### start handle FILL ####
#### start handle MLPY ####
#### start handle DRIP ####
#### start handle PXE ####
#### start handle YMLP ####
#### start handle CRAK ####
#### start handle IMLP ####
#### start handle GASL ####
#### start handle MLPO ####
#### start

In [14]:
pd.read_pickle('df_all.h5')

,Date,XLE,AMLP,VDE,AMJ,EMLP,XOP,MLPI,IXC,OIH,...,YLCO,PUW,DUG,ENY,MLPG,MLPB,AMUB,GASX,CHIE,DDG
0,2015-12-31,54.83,9.26,75.92,23.11,17.89,29.40,20.59,25.54,24.85,...,9.20,18.90,71.58,6.12,18.16,17.70,12.76,212.19,9.20,30.93
1,2016-01-04,54.82,9.34,75.91,23.42,17.96,29.55,20.83,25.45,25.10,...,9.06,18.91,71.74,6.05,17.99,17.70,12.76,209.28,9.20,31.14
2,2016-01-05,55.03,9.28,76.18,23.31,18.04,29.45,20.77,25.46,24.66,...,9.01,18.91,71.09,6.02,17.97,17.70,12.76,215.07,8.99,31.04
3,2016-01-06,52.91,8.75,73.22,22.17,17.63,27.43,19.63,24.57,23.55,...,8.89,18.09,76.45,5.80,16.91,17.70,12.76,271.50,8.99,32.11
4,2016-01-07,51.62,8.28,71.40,21.07,17.18,26.74,18.52,23.87,22.86,...,8.58,17.38,80.29,5.54,16.21,17.70,13.35,304.35,8.99,32.68
5,2016-01-08,50.95,8.44,70.49,21.32,17.35,26.87,18.82,23.57,22.37,...,8.63,17.24,82.35,5.61,16.15,18.89,13.35,278.99,8.63,32.91
6,2016-01-11,49.86,7.99,68.89,20.22,17.08,25.82,17.84,23.14,21.91,...,8.47,17.05,85.85,5.44,15.33,19.29,13.35,333.44,8.19,33.85
7,2016-01-12,49.98,7.96,69.06,20.23,16.90,25.41,17.82,23.14,21.85,...,8.41,16.70,85.27,5.40,15.05,19.29,13.35,379.67,8.35,33.99
8,2016-01-13,48.78,7.46,67.63,19.03,16.44,24.55,16.75,22.76,21.18,...,8.17,16.45,88.78,5.29,14.36,19.29,13.35,393.16,8.06,34.29
9,2016-01-14,50.88,7.96,70.57,20.15,16.97,25.40,17.78,23.77,21.89,...,8.32,16.78,81.16,5.49,15.01,19.29,13.35,356.66,8.07,32.88
